In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import random
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

In [2]:
# --- Configuration ---
MODEL_PATH = 'cifar10_cnn_model.pth'
PLOT_PATH = 'training_plots.png'
CONF_MATRIX_PATH = 'confusion_matrix.png'
REPORT_PATH = 'classification_report.txt'

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Hyperparameters
MAX_EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001
PATIENCE = 10

In [4]:
# Prepare dataset and dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

print("Downloading CIFAR-10 training dataset...")
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
# CNN Model Definition
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [6]:
def train_and_evaluate():
    model = SimpleCNN(num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

    best_loss = float('inf')
    epochs_no_improve = 0

    train_losses = []
    train_accuracies = []

    print("\n--- Starting Training ---")
    start_time = time.time()

    for epoch in range(MAX_EPOCHS):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        print(f"Epoch [{epoch+1}/{MAX_EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        scheduler.step(epoch_loss)

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), MODEL_PATH)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    elapsed = time.time() - start_time
    print(f"\nTraining completed in {elapsed:.2f} seconds.")

    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = accuracy_score(all_labels, all_preds)
    print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

    cls_report = classification_report(all_labels, all_preds, target_names=classes)
    print("\nClassification Report:\n", cls_report)

    with open(REPORT_PATH, 'w') as f:
        f.write(cls_report)

    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(CONF_MATRIX_PATH)
    plt.close()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig(PLOT_PATH)
    plt.close()

    print(f"Saved training plots to {PLOT_PATH}")
    print(f"Saved confusion matrix to {CONF_MATRIX_PATH}")
    print(f"Saved classification report to {REPORT_PATH}")

In [7]:
train_and_evaluate()


--- Starting Training ---
Epoch [1/100], Loss: 1.4556, Accuracy: 46.69%
Epoch [2/100], Loss: 1.0362, Accuracy: 63.24%
Epoch [3/100], Loss: 0.8556, Accuracy: 70.22%
Epoch [4/100], Loss: 0.7347, Accuracy: 74.18%
Epoch [5/100], Loss: 0.6537, Accuracy: 77.00%
Epoch [6/100], Loss: 0.5748, Accuracy: 79.84%
Epoch [7/100], Loss: 0.5149, Accuracy: 81.80%
Epoch [8/100], Loss: 0.4568, Accuracy: 83.77%
Epoch [9/100], Loss: 0.4055, Accuracy: 85.45%
Epoch [10/100], Loss: 0.3652, Accuracy: 87.09%
Epoch [11/100], Loss: 0.3278, Accuracy: 88.20%
Epoch [12/100], Loss: 0.2922, Accuracy: 89.46%
Epoch [13/100], Loss: 0.2708, Accuracy: 90.27%
Epoch [14/100], Loss: 0.2499, Accuracy: 91.01%
Epoch [15/100], Loss: 0.2335, Accuracy: 91.66%
Epoch [16/100], Loss: 0.2177, Accuracy: 92.10%
Epoch [17/100], Loss: 0.1999, Accuracy: 92.81%
Epoch [18/100], Loss: 0.1955, Accuracy: 92.92%
Epoch [19/100], Loss: 0.1845, Accuracy: 93.36%
Epoch [20/100], Loss: 0.1736, Accuracy: 93.90%
Epoch [21/100], Loss: 0.1685, Accuracy: 94